In [ ]:
!pip install neowise

In [ ]:
import neowise as nw
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from matplotlib import pyplot
from matplotlib import cm

In [ ]:
# Data Preparation --- Two Spirals
def twospirals(n_points, noise=0.5):
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), 
            np.hstack((np.zeros(n_points),np.ones(n_points))))

N_SAMPLES = 1000
X_data, y_data_t = twospirals(N_SAMPLES)
y_data = y_data_t.reshape(N_SAMPLES*2,1)
mu1 = np.mean(X_data[:,0],axis=0)
mu2 = np.mean(X_data[:,1],axis=0)
var1 = np.var(X_data[:,0],axis=0)
var2 = np.var(X_data[:,1],axis=0)
X_data[:,0] = (X_data[:,0]-mu1)/var1
X_data[:,1] = (X_data[:,1]-mu2)/var2
m = int(X_data.shape[0])
m_tr = int(math.ceil((90/100)*m))
m_cv = int(math.ceil((5/100)*m))
m_te = m - (m_tr + m_cv)
X_tr = np.zeros((m_tr,X_data.shape[1]))
y_tr_t = np.zeros((m_tr,1))
X_cv = np.zeros((m_cv,X_data.shape[1]))
y_cv_t = np.zeros((m_cv,1))
X_te = np.zeros((m_te,X_data.shape[1]))
y_te_t = np.zeros((m_te,1))
perm = np.random.permutation(m)
p = perm.reshape(m,1)
for i in range(0,m_tr):
  X_tr[i] = X_data[p[i]]
  y_tr_t[i] = y_data[p[i]]
for i in range(0,m_cv):
  X_cv[i] = X_data[p[i+m_tr]]
  y_cv_t[i] = y_data[p[i+m_tr]]
for i in range(0,m_te):
  X_te[i] = X_data[p[i+m_tr+m_cv]]
  y_te_t[i] = y_data[p[i+m_tr+m_cv]]
y_tr = y_tr_t.T
y_cv = y_cv_t.T
y_te = y_te_t.T
plt.title('Training Set')
plt.plot(X_data[y_data_t==0,0], X_data[y_data_t==0,1], '.', label='class 1')
plt.plot(X_data[y_data_t==1,0], X_data[y_data_t==1,1], '.', label='class 2')
plt.legend()
plt.show()

In [ ]:
model = nw.Model(X_tr,y_tr,X_te,y_te,X_cv,y_cv)

In [ ]:
model.reset()
model.add("dense1",X_tr.shape[1],500,"relu")
model.add("dense2",500,250,"relu")
model.add("dense3",250,150,"sine")
model.add("dense4",150,100,"sine")
model.add("dense5",100,60,"sine")
model.add("dense6",60,30,"relu")
model.add("dense7",30,15,"tanh")
model.add("dense8",15,1,"sigmoid")

In [ ]:
model.summary()

In [ ]:
model.fit(X_tr,y_tr,0.005,15,"Adam","Binary",mb=32,alpha_decay=True)

In [ ]:
model.test(X_te,y_te,"Binary")

In [ ]:
model.plot("Cost")

In [ ]:
model.plot("Accuracy")

## Replace contents of argument "direc" with the directory you want to save the images!

## For saving and loading the model, pass the directory in which you want to save the file as the argument and append it by the name of the file with .h5 extension!

In [ ]:
model.plot("Cost",animate=True,direc='/content/drive/My Drive/neowise/spirals/spirals-costs/')

In [ ]:
model.plot("Accuracy",animate=True,direc='/content/drive/My Drive/neowise/spirals/spirals-accu/')

In [ ]:
model.save_model('/content/drive/My Drive/neowise/spirals/spirals.h5')

In [ ]:
model1 = nw.Model(X_tr,y_tr,X_te,y_te,X_cv,y_cv)

In [ ]:
model1.load_model('/content/drive/My Drive/neowise/spirals/spirals.h5')

In [ ]:
model1.summary()

In [ ]:
model1.test(X_te,y_te,"Binary")

In [ ]:
sns.set_style("whitegrid")
GRID_X_START = 0.5
GRID_X_END = -0.5
GRID_Y_START = 0.5
GRID_Y_END = -0.5
OUTPUT_DIR = "/content/drive/My Drive/neowise/spirals/spirals-vis"
grid = np.mgrid[GRID_X_START:GRID_X_END:100j,GRID_X_START:GRID_Y_END:100j]
grid_2d = grid.reshape(2,-1)
XX, YY = grid
def make_plot(X, y, plot_name, file_name=None, XX=None, YY=None, preds=None, dark=False):
    if (dark):
        plt.style.use('dark_background')
    else:
        sns.set_style("whitegrid")
    plt.figure(figsize=(16,12))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if(XX is not None and YY is not None and preds is not None):
        plt.contourf(XX, YY, preds.reshape(XX.shape), 25, alpha = 1, cmap=cm.Spectral)
        plt.contour(XX, YY, preds.reshape(XX.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    if(file_name):
        plt.savefig(file_name)
        plt.close()
import os
def callback_numpy_plot(index, init_params):
    plot_title = "Iteration {:05}".format(index)
    file_name = "numpy_model_{:05}.png".format(index//1)
    file_path = os.path.join(OUTPUT_DIR, file_name)
    act = model.forward_prop(np.transpose(grid_2d),train_model=False)
    prediction_probs = act["A" + str(len(model.layer_names))]
    prediction_probs = prediction_probs.reshape(prediction_probs.shape[1], 1)
    make_plot(X_te, y_te, plot_title, file_name=file_path, XX=XX, YY=YY, preds=prediction_probs, dark=True)

In [ ]:
model.reset()
model.add("dense1",X_tr.shape[1],500,"relu")
model.add("dense2",500,250,"relu")
model.add("dense3",250,150,"sine")
model.add("dense4",150,100,"sine")
model.add("dense5",100,60,"sine")
model.add("dense6",60,30,"relu")
model.add("dense7",30,15,"tanh")
model.add("dense8",15,1,"sigmoid")

model.fit(X_tr,y_tr,0.005,15,"Adam","Binary",mb=32,alpha_decay=True,print_cost=False,callback=callback_numpy_plot)

In [ ]:
act = model.forward_prop(np.transpose(grid_2d),train_model=True)
prediction_probs_np = act["A" + str(len(model.layer_names))]
prediction_probs_np = prediction_probs_np.reshape(prediction_probs_np.shape[1], 1)
make_plot(X_te, y_te, "Final Iteration", file_name=None, XX=XX, YY=YY, preds=prediction_probs_np, dark=True)